In [ ]:
#Load libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from keras.optimizers import Adam

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import pickle



In [ ]:
# Import Data

train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

#train_df = train_df[0:200]
#test_df = test_df[0:200]
print(train_df.head())
print(f'Train Data Shape: {train_df.shape}')
print(f'Test Data Shape: {test_df.shape}')

In [ ]:
train_df.head()

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='target', data=train_df)
plt.title('Target Labels')
plt.xlabel('Target')
plt.ylabel('Count')
plt.show()

In [ ]:
train_df['text_length'] = train_df['text'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10, 6))
sns.histplot(train_df['text_length'], bins=30, kde=True)
plt.title('Distribution of Text Lengths')
plt.xlabel('Text Length')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(y='keyword', data=train_df, hue='target', order=train_df['keyword'].value_counts().iloc[:15].index)
plt.title('Top 15 Keywords by Target Label')
plt.xlabel('Count')
plt.ylabel('Keyword')
plt.legend(title='Target')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(y='location', data=train_df, hue='target', order=train_df['location'].value_counts().iloc[:15].index)
plt.title('Top 15 Locations by Target Label')
plt.xlabel('Count')
plt.ylabel('Location')
plt.legend(title='Target')
plt.show()


In [ ]:
from wordcloud import WordCloud

disaster_tweets = train_df[train_df['target'] == 1]['text']
non_disaster_tweets = train_df[train_df['target'] == 0]['text']

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
wordcloud_disaster = WordCloud(width=800, height=400, background_color='white').generate(' '.join(disaster_tweets))
plt.imshow(wordcloud_disaster, interpolation='bilinear')
plt.title('Disaster Tweets')
plt.axis('off')

plt.subplot(1, 2, 2)
wordcloud_non_disaster = WordCloud(width=800, height=400, background_color='white').generate(' '.join(non_disaster_tweets))
plt.imshow(wordcloud_non_disaster, interpolation='bilinear')
plt.title('Non-Disaster Tweets')
plt.axis('off')

plt.show()

In [ ]:
modes_keyword = train_df["keyword"].mode()[0] # Get Mode of zeroth value
modes_location = test_df["location"].mode()[0] # Get Mode of zeroth value
print(f'modes_keyword: {modes_keyword}')
print(f'modes_location: {modes_location}')

train_df["keyword"] = train_df["keyword"].fillna(modes_keyword) # Assign
train_df["location"] = train_df["location"].fillna(modes_location) # Assign

test_df["keyword"] = test_df["keyword"].fillna(modes_keyword) # Assign
test_df["location"] = test_df["location"].fillna(modes_location) # Assign

In [ ]:
max_w = 10000
tokenizer = Tokenizer(num_words = max_w,oov_token = '<OOV>')
tokenizer.fit_on_texts(train_df["text"])

In [ ]:
X_train, y_test = train_test_split(train_df, test_size = 0.2, random_state = 0)
print(f'y_test: {len(y_test)}')
print(f'test_df: {len(test_df)}')

In [ ]:
max_length = 200
train_seq = tokenizer.texts_to_sequences(X_train["text"])
train_padded = pad_sequences(train_seq, maxlen = max_length, padding = "post", truncating = "post")


In [ ]:
test_seq = tokenizer.texts_to_sequences(y_test["text"])
test_padded = pad_sequences(test_seq, maxlen = max_length, padding = "post", truncating = "post")
print(f'Test Padded; {test_padded}')

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max', restore_best_weights=True)

In [ ]:
s_model = Sequential()
s_model.add(Embedding(input_dim=max_w, output_dim=128))
s_model.add(LSTM(64, return_sequences=True))
#s_model.add(LSTM(64, return_sequences=True))
s_model.add(Dropout(0.2))
s_model.add(Dense(1, activation='sigmoid'))
s_model.add(Flatten())  # Flatten layer to 2D
#s_model.add(Dense(64, activation = 'relu'))
s_model.add(Dense(1, activation = 'relu'))
s_model.add(Dropout(0.2))
#s_model.add(Dense(32, activation = 'relu'))
s_model.add(Dense(1, activation = 'sigmoid'))
print("Model Ready!")

#flatten_layer = Flatten()(conv_output)
#dense_layer = Dense(12800, activation='relu')#(flatten_layer)
#s_model.add(dense_layer)

#model = tf.keras.Sequential([
#  layers.Embedding(max_features, embedding_dim),
#  layers.Dropout(0.2),
#  layers.GlobalAveragePooling1D(),
#  layers.Dropout(0.2),
#  layers.Dense(1, activation='sigmoid')])

In [ ]:
s_model.summary()

In [ ]:
s_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print(f"y_test['target']: \n{y_test['target']}")

In [ ]:

history = s_model.fit(
    train_padded, 
    X_train['target'],
    epochs = 10, 
    batch_size=256, 
    validation_data=(test_padded, y_test['target']), 
    callbacks=[early_stopping]
)

#s_model.fit(train_padded, epochs=1, validation_data=test_generator)



In [ ]:
plt.figure(figsize=(6,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.ylim(0.8, 1)
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
test_seq = tokenizer.texts_to_sequences(test_df["text"])
test_padded = pad_sequences(test_seq, maxlen=max_length, padding="post", truncating="post")


predictions = s_model.predict(test_padded)
print(f'Predictions: {predictions}')
binary_predictions = (predictions > 0.5).astype(int)

test_df['target'] = binary_predictions
print(test_df[['id', 'text', 'target']])

In [ ]:
# Predict on validation data
y_pred = s_model.predict(test_padded)
print(y_pred)
y_pred = (y_pred > 0.5).astype(int)

# Calculate F1-score
f1 = f1_score(test_df['target'], y_pred)
print(f"F1 Score: {f1}")

In [ ]:
print(y_pred)

In [ ]:
y_pred_flattened = y_pred.flatten()
output = pd.DataFrame(
    {
    'id': test_df.id, 
    'target': y_pred_flattened
    }
)
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
#test_df.to_csv('submission.csv', index=False)

In [ ]:
#sample_submission["target"] = np.argmax(s_model.predict(X_test), axis=1)